In [1]:
from datasets import load_dataset
# Pip install urllib==1.26.7

import pandas as pd 

In [2]:
dataset = load_dataset("mteb/tweet_sentiment_extraction") 
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 27481
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 3534
    })
})

In [3]:
df = pd.DataFrame(dataset['train'])

In [4]:
df.head()

,id,text,label,label_text
0,cb774db0d1,"I`d have responded, if I were going",1,neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,0,negative
2,088c60f138,my boss is bullying me...,0,negative
3,9642c003ef,what interview! leave me alone,0,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...",0,negative


In [5]:
# Tokenizer 
from transformers import GPT2Tokenizer 

tokenizer = GPT2Tokenizer.from_pretrained("gpt2") 
tokenizer.pad_token = tokenizer.eos_token

def tokenizer_function(examples): 
    return tokenizer(examples['text'], padding='max_length', truncation=True) 

tokenized_datasets = dataset.map(tokenizer_function, batched=True)

c:\Users\Windows\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
seed = 42 
small_train_dataset = tokenized_datasets['train'].shuffle(seed=seed).select(range(1000)) 
small_eval_dataset = tokenized_datasets['test'].shuffle(seed=seed).select(range(1000))

In [7]:
from transformers import GPT2ForSequenceClassification 
model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=3)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
import evaluate 
metric = evaluate.load("hyperml/balanced_accuracy") 

def compute_metrics(eval_pred): 
    logits, labels = eval_pred 
    predictions = np.argmax(logits, axis=-1) 
    return metric.compute(predictions=predictions, references=labels)

In [9]:
from transformers import TrainingArguments, Trainer 

training_args = TrainingArguments(
    output_dir="test_trainer", 
    evaluation_strategy="epoch", 
    per_device_train_batch_size=1, 
    per_device_eval_batch_size=1, 
    gradient_accumulation_steps=4
)

trainer = Trainer(
    model=model, 
    args=training_args,
    train_dataset=small_train_dataset, 
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

c:\Users\Windows\anaconda3\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/750 [00:00<?, ?it/s]

In [ ]:
# Evaluate 
trainer.evaluate() 